In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
# train_set = pd.read_csv('./train_set_onehot.csv')
# test_set = pd.read_csv('./test_set.csv')
# test_set = test_set.rename(columns={'due_amt':'due_amt_scale'})

# train_set['due_amt_scale'] = train_set['due_amt']
# scale_cols = ['due_amt_scale', 'auditing_date_month', 'auditing_date_days',
#                'auditing_date_dayofweek', 'due_date_month', 'due_date_dayofweek',
#                 'rate', 'principal', 'pvi','audit_reg_delta','tag_num', 'behavior_time_month', 
#                   'behavior_time_days','behavior_time_dayofweek', 'same_behavior_count1d', 'audit_beh_delta']

# # minmaxscaler = MinMaxScaler()
# # minmaxscaler.fit(train_set[scale_cols])
# # train_set[scale_cols] = minmaxscaler.tranform(train_set[scale_cols])
# for col in scale_cols:
#     desc = train_set[col].describe()
#     train_set[col] = (train_set[col]-desc['mean'])/desc['std']
# #     try:
#     test_set[col] = (test_set[col]-desc['mean'])/desc['std']
# #     except:
# #         pass
# train_set.columns

# # train_set = train_set[['repay_amt','due_amt','due_amt_scale']+list(scale_cols.drop(['due_amt_scale']))]
# train_set.to_csv('./train_set_onehot_scaler.csv',index=False)
# test_set.to_csv('./test_set_onehot_scaler.csv',index=False)

In [3]:
learning_rate = 0.05
global_step = tf.Variable(0,trainable=False)
decaylearning_rate = tf.train.exponential_decay(learning_rate,global_step,1000,0.8)

n_input = 74
n_classes = 33

n_hidden_1 = 128
n_hidden_2 = 256
n_hidden_3 = 128

x = tf.placeholder(tf.float32,[None,n_input],name='x')
y = tf.placeholder(tf.float32,[None,n_classes],name='y')
y_ = tf.placeholder(tf.float32,[None,n_classes],name='y_')

repay_amt=tf.placeholder(tf.float32,[None,],name='repay_amt')
due_amt=tf.placeholder(tf.float32,[None,],name='due_amt')

drop_rate_1 = 0.1
drop_rate_2 = 0.2
drop_rate_3 = 0.1

l2_decay=0.0001

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
weights = {
    'h1': tf.Variable(tf.truncated_normal([n_input,n_hidden_1])),
    'h2': tf.Variable(tf.truncated_normal([n_hidden_1,n_hidden_2])),
    'h3': tf.Variable(tf.truncated_normal([n_hidden_2,n_hidden_3])),
    'out': tf.Variable(tf.truncated_normal([n_hidden_3,n_classes]))
}

biases = {
    'b1':tf.Variable(tf.truncated_normal([n_hidden_1])),
    'b2':tf.Variable(tf.truncated_normal([n_hidden_2])),
    'b3':tf.Variable(tf.truncated_normal([n_hidden_3])),
    'out':tf.Variable(tf.truncated_normal([n_classes]))
}

def multilayer_perceptron(x,weights,biases):
    layer_1 = tf.add(tf.matmul(x,weights['h1']),biases['b1'])
    layer_1 = tf.nn.leaky_relu(layer_1)
    layer_1_drop = tf.nn.dropout(layer_1,rate=drop_rate_1)
    
    layer_2 = tf.add(tf.matmul(layer_1,weights['h2']),biases['b2'])
    layer_2 = tf.nn.leaky_relu(layer_2)
    layer_2_drop = tf.nn.dropout(layer_2,rate=drop_rate_2)
    
    layer_3 = tf.add(tf.matmul(layer_2,weights['h3']),biases['b3'])
    layer_3 = tf.nn.leaky_relu(layer_3)
    layer_3_drop = tf.nn.dropout(layer_3,rate=drop_rate_3)
    
    out_layer = tf.add(tf.matmul(layer_3,weights['out']),biases['out'],name='output')
    
    return out_layer

pred = multilayer_perceptron(x,weights,biases)

In [6]:
# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))

y_pred = tf.nn.softmax(logits=pred)
y_pred = tf.clip_by_value(y_pred, 1e-10, 1.0 - 1e-10)
cost = tf.reduce_mean(-y * ((1 - y_pred) ** 2.0) * tf.log(y_pred))

l2_regular=l2_decay*tf.add_n([tf.nn.l2_loss(var) for var in tf.trainable_variables()])
rmse=tf.sqrt(tf.reduce_mean(tf.pow(tf.reduce_sum(tf.nn.softmax(pred)*y_,-1)*due_amt-repay_amt,2)))
total_loss=cost+l2_regular

correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

optimizer = tf.train.AdamOptimizer(decaylearning_rate).minimize(total_loss,global_step=global_step)

tf.summary.scalar('cost',cost)
tf.summary.scalar('l2_regular',l2_regular)
tf.summary.scalar('rmse',rmse)
tf.summary.scalar('total_loss',total_loss)
summary=tf.summary.merge_all()

In [7]:
training_epochs = 25
batch_size = 1000
file='./train_set_onehot.csv'

In [8]:
saver=tf.train.Saver(max_to_keep=1)
init = tf.global_variables_initializer()

In [9]:
with tf.Session() as sess:
    sess.run(init)
    summary_writer=tf.summary.FileWriter('../tensorboard_summary',sess.graph)
#     model_ckpt = tf.train.latest_checkpoint('../save_models/')
#     print(model_ckpt)
#     saver.restore(sess,model_ckpt)

    for epoch in range(training_epochs):
        batches = pd.read_csv(file,chunksize=batch_size)
        total_batch = 0
        for batch in batches:
            total_batch +=1
#             print(batch.iloc[:,:].info())
            input_features = batch.iloc[:,1:75]
            labels = batch.iloc[:,75:].values
#             print(labels.shape)
            labels_ = labels.copy()
            labels_[:,32] = 0.
#             print(labels_.shape)
            repay_amt_ = batch.iloc[:,0].values
            due_amt_ = batch.iloc[:,1].values
            
            _,cost_,l2_regular_,rmse_,total_loss_,acc,global_step_,merge_summary=sess.run(
                            [optimizer,cost,l2_regular,rmse,total_loss,accuracy,global_step,summary],
                            feed_dict= { x:input_features, y:labels, y_:labels_,
                                        repay_amt:repay_amt_, due_amt:due_amt_})
            summary_writer.add_summary(merge_summary,global_step_)
            if global_step_%100==0:
                print('[epoch %s][step %s]' % (epoch,global_step_),
                     'logloss:%.6f' % cost_,
                     'l2_regular:%.6f' % l2_regular_,
                     'rmse:%.6f' % rmse_,
                     'total_loss:%.6f' % total_loss_,
                     'acc:%.6f' % acc)
            
            if global_step_%1000==0:
                saver.save(sess,"../save_models/mlp.ckpt",global_step=global_step_)
        print('--------------epoch %s finished —> total batch:%s---------------' % (epoch,total_batch))
    print('finished!')

[epoch 0][step 100] logloss:0.586810 l2_regular:0.015767 rmse:396.786774 total_loss:0.602577 acc:0.159000
[epoch 0][step 200] logloss:0.669843 l2_regular:0.001650 rmse:482.090576 total_loss:0.671492 acc:0.040000
[epoch 0][step 300] logloss:0.631467 l2_regular:0.002816 rmse:598.264526 total_loss:0.634283 acc:0.095000
[epoch 0][step 400] logloss:0.500289 l2_regular:0.001050 rmse:345.478760 total_loss:0.501339 acc:0.283000
[epoch 0][step 500] logloss:0.568669 l2_regular:0.000807 rmse:381.914948 total_loss:0.569476 acc:0.185000
[epoch 0][step 600] logloss:0.697753 l2_regular:0.283959 rmse:407.686584 total_loss:0.981712 acc:0.000000
[epoch 0][step 700] logloss:0.694962 l2_regular:0.066983 rmse:510.595306 total_loss:0.761945 acc:0.004000
[epoch 0][step 800] logloss:0.666354 l2_regular:0.034607 rmse:626.535400 total_loss:0.700961 acc:0.045000
[epoch 0][step 900] logloss:0.697753 l2_regular:0.021919 rmse:585.164124 total_loss:0.719672 acc:0.000000
[epoch 0][step 1000] logloss:0.692869 l2_regul

[epoch 0][step 7500] logloss:0.674029 l2_regular:0.003961 rmse:574.348572 total_loss:0.677991 acc:0.034000
[epoch 0][step 7600] logloss:0.575646 l2_regular:0.013774 rmse:473.534973 total_loss:0.589420 acc:0.175000
[epoch 0][step 7700] logloss:0.627978 l2_regular:0.010708 rmse:604.558594 total_loss:0.638686 acc:0.100000
[epoch 0][step 7800] logloss:0.639840 l2_regular:0.008834 rmse:845.780457 total_loss:0.648673 acc:0.083000
[epoch 0][step 7900] logloss:0.651701 l2_regular:0.007514 rmse:765.330200 total_loss:0.659215 acc:0.066000
[epoch 0][step 8000] logloss:0.625884 l2_regular:0.006515 rmse:437.908783 total_loss:0.632399 acc:0.103000
[epoch 0][step 8100] logloss:0.628676 l2_regular:0.005723 rmse:463.584045 total_loss:0.634399 acc:0.099000
[epoch 0][step 8200] logloss:0.610534 l2_regular:0.005076 rmse:432.016205 total_loss:0.615610 acc:0.125000
[epoch 0][step 8300] logloss:0.628676 l2_regular:0.004535 rmse:387.095367 total_loss:0.633211 acc:0.099000
--------------epoch 0 finished —> tot

[epoch 1][step 15100] logloss:0.637049 l2_regular:0.003844 rmse:579.736938 total_loss:0.640892 acc:0.087000
[epoch 1][step 15200] logloss:0.634955 l2_regular:0.003204 rmse:505.737305 total_loss:0.638159 acc:0.090000
[epoch 1][step 15300] logloss:0.644026 l2_regular:0.002747 rmse:471.862671 total_loss:0.646773 acc:0.077000
[epoch 1][step 15400] logloss:0.628676 l2_regular:0.002398 rmse:514.526062 total_loss:0.631074 acc:0.099000
[epoch 1][step 15500] logloss:0.580531 l2_regular:0.002122 rmse:536.923279 total_loss:0.582652 acc:0.168000
[epoch 1][step 15600] logloss:0.577740 l2_regular:0.001895 rmse:563.116272 total_loss:0.579635 acc:0.172000
[epoch 1][step 15700] logloss:0.643328 l2_regular:0.001706 rmse:597.243042 total_loss:0.645034 acc:0.078000
[epoch 1][step 15800] logloss:0.668447 l2_regular:0.001544 rmse:804.056152 total_loss:0.669992 acc:0.042000
[epoch 1][step 15900] logloss:0.625884 l2_regular:0.001405 rmse:502.379944 total_loss:0.627289 acc:0.103000
[epoch 1][step 16000] loglos

[epoch 2][step 22700] logloss:0.064534 l2_regular:0.000830 rmse:468.563110 total_loss:0.065364 acc:0.218000
[epoch 2][step 22800] logloss:0.064839 l2_regular:0.000828 rmse:418.296814 total_loss:0.065667 acc:0.194000
[epoch 2][step 22900] logloss:0.065191 l2_regular:0.000830 rmse:412.724854 total_loss:0.066021 acc:0.242000
[epoch 2][step 23000] logloss:0.062639 l2_regular:0.000816 rmse:452.251190 total_loss:0.063455 acc:0.170000
[epoch 2][step 23100] logloss:0.058420 l2_regular:0.000812 rmse:511.651184 total_loss:0.059232 acc:0.245000
[epoch 2][step 23200] logloss:0.069384 l2_regular:0.000804 rmse:405.855469 total_loss:0.070188 acc:0.305000
[epoch 2][step 23300] logloss:0.067139 l2_regular:0.000806 rmse:596.757507 total_loss:0.067945 acc:0.269000
[epoch 2][step 23400] logloss:0.049830 l2_regular:0.000803 rmse:344.729553 total_loss:0.050634 acc:0.285000
[epoch 2][step 23500] logloss:0.067476 l2_regular:0.000809 rmse:376.499847 total_loss:0.068285 acc:0.201000
[epoch 2][step 23600] loglos

[epoch 3][step 30300] logloss:0.055889 l2_regular:0.000635 rmse:414.817078 total_loss:0.056524 acc:0.259000
[epoch 3][step 30400] logloss:0.052537 l2_regular:0.000634 rmse:420.348877 total_loss:0.053171 acc:0.231000
[epoch 3][step 30500] logloss:0.058462 l2_regular:0.000633 rmse:550.691467 total_loss:0.059094 acc:0.223000
[epoch 3][step 30600] logloss:0.055548 l2_regular:0.000632 rmse:351.753265 total_loss:0.056180 acc:0.308000
[epoch 3][step 30700] logloss:0.076586 l2_regular:0.000629 rmse:491.221680 total_loss:0.077215 acc:0.134000
[epoch 3][step 30800] logloss:0.059195 l2_regular:0.000627 rmse:477.698792 total_loss:0.059821 acc:0.353000
[epoch 3][step 30900] logloss:0.050011 l2_regular:0.000627 rmse:470.596588 total_loss:0.050638 acc:0.355000
[epoch 3][step 31000] logloss:0.069186 l2_regular:0.000626 rmse:470.835693 total_loss:0.069812 acc:0.221000
[epoch 3][step 31100] logloss:0.050054 l2_regular:0.000625 rmse:549.355164 total_loss:0.050679 acc:0.369000
[epoch 3][step 31200] loglos

...

[epoch 20][step 172700] logloss:0.056879 l2_regular:0.000608 rmse:521.337830 total_loss:0.057487 acc:0.236000
[epoch 20][step 172800] logloss:0.050015 l2_regular:0.000608 rmse:393.505463 total_loss:0.050623 acc:0.287000
[epoch 20][step 172900] logloss:0.048299 l2_regular:0.000608 rmse:592.101501 total_loss:0.048907 acc:0.224000
[epoch 20][step 173000] logloss:0.060819 l2_regular:0.000608 rmse:430.884949 total_loss:0.061427 acc:0.255000
[epoch 20][step 173100] logloss:0.071460 l2_regular:0.000608 rmse:489.200317 total_loss:0.072067 acc:0.268000
[epoch 20][step 173200] logloss:0.061753 l2_regular:0.000608 rmse:537.317993 total_loss:0.062361 acc:0.166000
[epoch 20][step 173300] logloss:0.063946 l2_regular:0.000608 rmse:434.105469 total_loss:0.064554 acc:0.256000
[epoch 20][step 173400] logloss:0.066626 l2_regular:0.000608 rmse:581.753357 total_loss:0.067234 acc:0.211000
[epoch 20][step 173500] logloss:0.065521 l2_regular:0.000608 rmse:385.755890 total_loss:0.066129 acc:0.225000
[epoch 20]

[epoch 21][step 180100] logloss:0.054161 l2_regular:0.000608 rmse:404.884888 total_loss:0.054769 acc:0.301000
[epoch 21][step 180200] logloss:0.050490 l2_regular:0.000608 rmse:708.550598 total_loss:0.051097 acc:0.342000
[epoch 21][step 180300] logloss:0.063634 l2_regular:0.000608 rmse:583.965942 total_loss:0.064242 acc:0.240000
[epoch 21][step 180400] logloss:0.057437 l2_regular:0.000608 rmse:457.010132 total_loss:0.058044 acc:0.232000
[epoch 21][step 180500] logloss:0.056007 l2_regular:0.000608 rmse:377.888611 total_loss:0.056615 acc:0.194000
[epoch 21][step 180600] logloss:0.065423 l2_regular:0.000608 rmse:373.896606 total_loss:0.066031 acc:0.223000
[epoch 21][step 180700] logloss:0.065795 l2_regular:0.000608 rmse:393.818359 total_loss:0.066403 acc:0.244000
[epoch 21][step 180800] logloss:0.071415 l2_regular:0.000608 rmse:429.477844 total_loss:0.072023 acc:0.168000
[epoch 21][step 180900] logloss:0.054162 l2_regular:0.000608 rmse:421.070190 total_loss:0.054770 acc:0.309000
[epoch 21]

[epoch 22][step 187500] logloss:0.062089 l2_regular:0.000608 rmse:427.016510 total_loss:0.062697 acc:0.195000
[epoch 22][step 187600] logloss:0.072559 l2_regular:0.000608 rmse:457.169617 total_loss:0.073167 acc:0.177000
[epoch 22][step 187700] logloss:0.061004 l2_regular:0.000608 rmse:505.149353 total_loss:0.061612 acc:0.272000
[epoch 22][step 187800] logloss:0.058112 l2_regular:0.000608 rmse:434.001678 total_loss:0.058719 acc:0.268000
[epoch 22][step 187900] logloss:0.058223 l2_regular:0.000608 rmse:354.793671 total_loss:0.058831 acc:0.273000
[epoch 22][step 188000] logloss:0.063078 l2_regular:0.000608 rmse:540.202271 total_loss:0.063686 acc:0.173000
[epoch 22][step 188100] logloss:0.059706 l2_regular:0.000608 rmse:495.885040 total_loss:0.060314 acc:0.234000
[epoch 22][step 188200] logloss:0.061514 l2_regular:0.000608 rmse:429.176758 total_loss:0.062121 acc:0.264000
[epoch 22][step 188300] logloss:0.057480 l2_regular:0.000608 rmse:622.629456 total_loss:0.058088 acc:0.389000
[epoch 22]

[epoch 23][step 194900] logloss:0.064048 l2_regular:0.000608 rmse:599.022339 total_loss:0.064656 acc:0.295000
[epoch 23][step 195000] logloss:0.059200 l2_regular:0.000608 rmse:461.220886 total_loss:0.059808 acc:0.293000
[epoch 23][step 195100] logloss:0.075285 l2_regular:0.000608 rmse:451.614960 total_loss:0.075892 acc:0.220000
[epoch 23][step 195200] logloss:0.065413 l2_regular:0.000608 rmse:510.219971 total_loss:0.066020 acc:0.226000
[epoch 23][step 195300] logloss:0.059982 l2_regular:0.000608 rmse:620.759521 total_loss:0.060590 acc:0.178000
[epoch 23][step 195400] logloss:0.065678 l2_regular:0.000608 rmse:401.746918 total_loss:0.066286 acc:0.201000
[epoch 23][step 195500] logloss:0.061467 l2_regular:0.000608 rmse:423.351624 total_loss:0.062074 acc:0.219000
[epoch 23][step 195600] logloss:0.059186 l2_regular:0.000608 rmse:425.268250 total_loss:0.059794 acc:0.212000
[epoch 23][step 195700] logloss:0.052002 l2_regular:0.000608 rmse:388.428986 total_loss:0.052610 acc:0.384000
[epoch 23]

[epoch 24][step 202300] logloss:0.058472 l2_regular:0.000608 rmse:383.853210 total_loss:0.059080 acc:0.204000
[epoch 24][step 202400] logloss:0.063454 l2_regular:0.000608 rmse:405.922943 total_loss:0.064061 acc:0.224000
[epoch 24][step 202500] logloss:0.048758 l2_regular:0.000608 rmse:351.647308 total_loss:0.049366 acc:0.249000
[epoch 24][step 202600] logloss:0.059433 l2_regular:0.000608 rmse:365.028137 total_loss:0.060041 acc:0.187000
[epoch 24][step 202700] logloss:0.067685 l2_regular:0.000608 rmse:543.453491 total_loss:0.068293 acc:0.210000
[epoch 24][step 202800] logloss:0.061313 l2_regular:0.000608 rmse:430.921906 total_loss:0.061921 acc:0.309000
[epoch 24][step 202900] logloss:0.055006 l2_regular:0.000608 rmse:344.461517 total_loss:0.055613 acc:0.349000
[epoch 24][step 203000] logloss:0.058890 l2_regular:0.000608 rmse:428.136993 total_loss:0.059498 acc:0.180000
[epoch 24][step 203100] logloss:0.069410 l2_regular:0.000608 rmse:489.270142 total_loss:0.070017 acc:0.194000
[epoch 24]

In [12]:
# import tensorflow as tf
import numpy as np
import glob

test_set = pd.read_csv('./test_set.csv')
test_set = test_set.iloc[:,2:]
model_path = "../save_models/mlp.ckpt-208000"

with tf.Session() as sess:

    saver = tf.train.import_meta_graph(model_path + ".meta")
    saver.restore(sess, model_path)
    test_data= test_set.values
    inputs = sess.graph.get_tensor_by_name('x:0')
    output = sess.graph.get_tensor_by_name('output:0')
    output=tf.nn.softmax(output)
    prob = sess.run([output], feed_dict={inputs: test_data})
    prob=prob[0]
    print(prob)

INFO:tensorflow:Restoring parameters from ../save_models/mlp.ckpt-208000
[[0.0036499  0.0022646  0.00171818 ... 0.24219659 0.243577   0.07169892]
 [0.00201436 0.00127222 0.00101843 ... 0.26661873 0.2682285  0.06204899]
 [0.00257524 0.00164869 0.00132895 ... 0.2536152  0.2531959  0.06141805]
 ...
 [0.00681111 0.0046021  0.00380963 ... 0.20500575 0.19352931 0.09350574]
 [0.00898046 0.00614844 0.00509679 ... 0.18420663 0.17242315 0.08453646]
 [0.00668033 0.00397577 0.00280746 ... 0.21763565 0.23722588 0.2131144 ]]


In [26]:
sub_info = pd.read_csv('../data_preprocess/sub_info.csv',parse_dates=['auditing_date'])
sub_test = sub_info[sub_info['repay_amt'].isnull()][['user_id','listing_id','auditing_date','due_amt']]
sub_test = sub_test.reset_index(drop=True)
print(sub_test.shape)
sub_test.head()

(786845, 4)


,user_id,listing_id,auditing_date,due_amt
0,498765,5431438,2019-03-12,138.5903
1,34524,5443211,2019-03-15,208.0805
2,34524,5443211,2019-03-15,208.0805
3,34524,5443211,2019-03-15,208.0805
4,34524,5443211,2019-03-15,208.0805


In [27]:
pred = prob

In [28]:
prob_cols = ['prob_%s' % i for i in range(pred.shape[1])]
pred_df = pd.concat([sub_test,pd.DataFrame(pred,columns=prob_cols)],axis=1)

In [29]:
pred_df.shape

(786845, 37)

In [30]:
pred_df = pred_df.drop_duplicates()
pred_df.shape

(786445, 37)

In [31]:
pred_df.head()

,user_id,listing_id,auditing_date,due_amt,prob_0,prob_1,prob_2,prob_3,prob_4,prob_5,...,prob_23,prob_24,prob_25,prob_26,prob_27,prob_28,prob_29,prob_30,prob_31,prob_32
0,498765,5431438,2019-03-12,138.5903,0.003650,0.002265,0.001718,0.001636,0.001779,0.001822,...,0.014920,0.019251,0.025081,0.032046,0.055460,0.085526,0.108246,0.242197,0.243577,0.071699
1,34524,5443211,2019-03-15,208.0805,0.002014,0.001272,0.001018,0.001008,0.001130,0.001148,...,0.012520,0.017017,0.021797,0.029373,0.052363,0.078141,0.115049,0.266619,0.268229,0.062049
2,34524,5443211,2019-03-15,208.0805,0.002575,0.001649,0.001329,0.001317,0.001471,0.001483,...,0.014274,0.019018,0.023963,0.031782,0.054975,0.079856,0.115032,0.253615,0.253196,0.061418
3,34524,5443211,2019-03-15,208.0805,0.002296,0.001458,0.001173,0.001161,0.001304,0.001313,...,0.013448,0.018116,0.022887,0.030677,0.053725,0.078102,0.116618,0.261568,0.258567,0.061199
4,34524,5443211,2019-03-15,208.0805,0.002095,0.001325,0.001060,0.001049,0.001174,0.001192,...,0.012776,0.017292,0.022128,0.029723,0.052781,0.078738,0.114888,0.264713,0.266287,0.061655


In [32]:
pred_df = pred_df.groupby(['user_id','listing_id','auditing_date','due_amt'])[prob_cols].mean().reset_index()

In [33]:
sub_example = pd.read_csv('../train_data/submission/submission.csv', parse_dates=['repay_date'])
sub_df = sub_example.merge(pred_df.drop('user_id',axis=1), on='listing_id', how='left')

In [35]:
sub_df['days'] = (sub_df['repay_date'] - sub_df['auditing_date']).dt.days
pred_prob = sub_df[prob_cols].values
pred_days = sub_df['days'].values
prob_someone_day = [pred_prob[i][pred_days[i]] for i in range(pred_prob.shape[0])]
sub_df['repay_amt'] = sub_df['due_amt'] * prob_someone_day


sub_df[['listing_id', 'repay_date', 'repay_amt']].to_csv('./predict_df_mlp.csv', index=False)